# Forecast Accuracy & Inventory Optimization

This notebook focuses on:
- Demand forecasting using ARIMA and Prophet
- Forecast accuracy measurement (MAPE)
- Inventory exposure risk analysis
- Safety stock optimization strategies

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.seasonal import seasonal_decompose
from prophet import Prophet
from sklearn.metrics import mean_absolute_error, mean_squared_error
import warnings
warnings.filterwarnings('ignore')

import sys
sys.path.append('..')
from utils import *

In [ ]:
# Load data
orders = pd.read_csv('../data/orders.csv', parse_dates=['order_date', 'planned_delivery', 'delivery_date'])
inventory = pd.read_csv('../data/inventory.csv')
products = pd.read_csv('../data/products.csv')

# Create demand time series
daily_demand = orders.groupby(['order_date', 'product_id'])['quantity'].sum().reset_index()
print(f"📊 Demand data: {len(daily_demand)} records across {daily_demand['product_id'].nunique()} products")

## 1. Demand Pattern Analysis

In [ ]:
# Aggregate daily demand across all products
total_daily_demand = orders.groupby('order_date')['quantity'].sum().reset_index()
total_daily_demand.set_index('order_date', inplace=True)

# Fill missing dates with zero demand
date_range = pd.date_range(start=total_daily_demand.index.min(), 
                          end=total_daily_demand.index.max(), 
                          freq='D')
total_daily_demand = total_daily_demand.reindex(date_range, fill_value=0)

print(f"📈 Total demand time series: {len(total_daily_demand)} days")
print(f"   Average daily demand: {total_daily_demand['quantity'].mean():.0f} units")
print(f"   Demand volatility (CV): {total_daily_demand['quantity'].std() / total_daily_demand['quantity'].mean():.2f}")

In [ ]:
# Visualize demand patterns
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Daily demand trend
total_daily_demand['quantity'].plot(ax=axes[0,0], title='Daily Demand Trend')
axes[0,0].set_ylabel('Quantity')

# Weekly seasonality
total_daily_demand['weekday'] = total_daily_demand.index.dayofweek
weekly_pattern = total_daily_demand.groupby('weekday')['quantity'].mean()
weekly_pattern.plot(kind='bar', ax=axes[0,1], title='Weekly Demand Pattern')
axes[0,1].set_xlabel('Day of Week (0=Monday)')
axes[0,1].set_ylabel('Avg Quantity')

# Monthly seasonality
monthly_demand = total_daily_demand.resample('M')['quantity'].sum()
monthly_demand.plot(ax=axes[1,0], title='Monthly Demand Trend')
axes[1,0].set_ylabel('Monthly Quantity')

# Demand distribution
axes[1,1].hist(total_daily_demand['quantity'], bins=30, alpha=0.7)
axes[1,1].set_title('Daily Demand Distribution')
axes[1,1].set_xlabel('Daily Quantity')
axes[1,1].set_ylabel('Frequency')

plt.tight_layout()
plt.show()

## 2. Time Series Decomposition

In [ ]:
# Perform seasonal decomposition
decomposition = seasonal_decompose(total_daily_demand['quantity'], 
                                  model='additive', 
                                  period=7)  # Weekly seasonality

# Plot decomposition
fig, axes = plt.subplots(4, 1, figsize=(15, 12))

decomposition.observed.plot(ax=axes[0], title='Original Demand')
decomposition.trend.plot(ax=axes[1], title='Trend Component')
decomposition.seasonal.plot(ax=axes[2], title='Seasonal Component')
decomposition.resid.plot(ax=axes[3], title='Residual Component')

for ax in axes:
    ax.grid(True)

plt.tight_layout()
plt.show()

# Calculate seasonality strength
seasonal_strength = 1 - (decomposition.resid.var() / decomposition.observed.var())
print(f"📊 Seasonality strength: {seasonal_strength:.3f}")

## 3. ARIMA Forecasting

In [ ]:
def arima_forecast(ts_data, forecast_periods=30, train_ratio=0.8):
    """Perform ARIMA forecasting with accuracy metrics"""
    
    # Split data
    train_size = int(len(ts_data) * train_ratio)
    train_data = ts_data[:train_size]
    test_data = ts_data[train_size:]
    
    # Fit ARIMA model
    model = ARIMA(train_data, order=(1, 1, 1))
    fitted_model = model.fit()
    
    # Generate forecasts
    forecast_test = fitted_model.forecast(steps=len(test_data))
    forecast_future = fitted_model.forecast(steps=forecast_periods)
    
    # Calculate accuracy metrics
    mape = calculate_mape(test_data, forecast_test)
    mae = mean_absolute_error(test_data, forecast_test)
    rmse = np.sqrt(mean_squared_error(test_data, forecast_test))
    
    return {
        'model': fitted_model,
        'train_data': train_data,
        'test_data': test_data,
        'forecast_test': forecast_test,
        'forecast_future': forecast_future,
        'mape': mape,
        'mae': mae,
        'rmse': rmse,
        'accuracy': 100 - mape
    }

# Run ARIMA forecast
arima_results = arima_forecast(total_daily_demand['quantity'])

print(f"🔮 ARIMA Forecast Results:")
print(f"   MAPE: {arima_results['mape']:.2f}%")
print(f"   Forecast Accuracy: {arima_results['accuracy']:.2f}%")
print(f"   MAE: {arima_results['mae']:.0f} units")
print(f"   RMSE: {arima_results['rmse']:.0f} units")

## 4. Prophet Forecasting

In [ ]:
def prophet_forecast(ts_data, forecast_periods=30, train_ratio=0.8):
    """Perform Prophet forecasting with accuracy metrics"""
    
    # Prepare data for Prophet
    prophet_data = ts_data.reset_index()
    prophet_data.columns = ['ds', 'y']
    
    # Split data
    train_size = int(len(prophet_data) * train_ratio)
    train_data = prophet_data[:train_size]
    test_data = prophet_data[train_size:]
    
    # Fit Prophet model
    model = Prophet(
        daily_seasonality=True,
        weekly_seasonality=True,
        yearly_seasonality=False,  # Not enough data for yearly
        changepoint_prior_scale=0.05
    )
    model.fit(train_data)
    
    # Generate forecasts
    future_test = model.make_future_dataframe(periods=len(test_data), include_history=False)
    forecast_test = model.predict(future_test)
    
    future_forecast = model.make_future_dataframe(periods=forecast_periods, include_history=False)
    forecast_future = model.predict(future_forecast)
    
    # Calculate accuracy metrics
    mape = calculate_mape(test_data['y'], forecast_test['yhat'])
    mae = mean_absolute_error(test_data['y'], forecast_test['yhat'])
    rmse = np.sqrt(mean_squared_error(test_data['y'], forecast_test['yhat']))
    
    return {
        'model': model,
        'train_data': train_data,
        'test_data': test_data,
        'forecast_test': forecast_test,
        'forecast_future': forecast_future,
        'mape': mape,
        'mae': mae,
        'rmse': rmse,
        'accuracy': 100 - mape
    }

# Run Prophet forecast
prophet_results = prophet_forecast(total_daily_demand['quantity'])

print(f"🔮 Prophet Forecast Results:")
print(f"   MAPE: {prophet_results['mape']:.2f}%")
print(f"   Forecast Accuracy: {prophet_results['accuracy']:.2f}%")
print(f"   MAE: {prophet_results['mae']:.0f} units")
print(f"   RMSE: {prophet_results['rmse']:.0f} units")

## 5. Model Comparison & Visualization

In [ ]:
# Compare model performance
model_comparison = pd.DataFrame({
    'Model': ['ARIMA', 'Prophet'],
    'MAPE': [arima_results['mape'], prophet_results['mape']],
    'Accuracy': [arima_results['accuracy'], prophet_results['accuracy']],
    'MAE': [arima_results['mae'], prophet_results['mae']],
    'RMSE': [arima_results['rmse'], prophet_results['rmse']]
})

print("📊 Model Comparison:")
print(model_comparison.round(2))

# Select best model
best_model = 'Prophet' if prophet_results['accuracy'] > arima_results['accuracy'] else 'ARIMA'
best_results = prophet_results if best_model == 'Prophet' else arima_results
print(f"\n🏆 Best Model: {best_model} (Accuracy: {best_results['accuracy']:.2f}%)")

In [ ]:
# Visualize forecasts
fig, axes = plt.subplots(2, 1, figsize=(15, 10))

# ARIMA forecast plot
train_dates = arima_results['train_data'].index
test_dates = arima_results['test_data'].index
future_dates = pd.date_range(start=test_dates[-1] + pd.Timedelta(days=1), periods=30, freq='D')

axes[0].plot(train_dates, arima_results['train_data'], label='Training Data', color='blue')
axes[0].plot(test_dates, arima_results['test_data'], label='Actual Test', color='green')
axes[0].plot(test_dates, arima_results['forecast_test'], label='ARIMA Forecast', color='red', linestyle='--')
axes[0].plot(future_dates, arima_results['forecast_future'], label='Future Forecast', color='orange', linestyle='--')
axes[0].set_title(f'ARIMA Forecast (Accuracy: {arima_results["accuracy"]:.1f}%)')
axes[0].legend()
axes[0].grid(True)

# Prophet forecast plot
prophet_results['model'].plot(prophet_results['forecast_future'], ax=axes[1])
axes[1].set_title(f'Prophet Forecast (Accuracy: {prophet_results["accuracy"]:.1f}%)')
axes[1].grid(True)

plt.tight_layout()
plt.show()

## 6. Inventory Exposure Risk Analysis

In [ ]:
# Calculate inventory exposure using forecast
def calculate_inventory_exposure(inventory_df, forecast_demand, forecast_horizon=30):
    """Calculate inventory exposure risk based on forecast"""
    
    exposure_analysis = inventory_df.copy()
    
    # Estimate demand for each product (simplified allocation)
    total_forecast = forecast_demand.sum()
    exposure_analysis['forecast_demand'] = exposure_analysis['avg_demand'] * forecast_horizon
    
    # Calculate exposure metrics
    exposure_analysis['days_of_supply'] = exposure_analysis['current_stock'] / exposure_analysis['avg_demand']
    exposure_analysis['stockout_risk'] = np.where(
        exposure_analysis['days_of_supply'] < forecast_horizon, 
        'High', 
        np.where(exposure_analysis['days_of_supply'] < forecast_horizon * 1.5, 'Medium', 'Low')
    )
    
    # Calculate overstock risk
    exposure_analysis['overstock_risk'] = np.where(
        exposure_analysis['current_stock'] > exposure_analysis['eoq'] * 2,
        'High',
        np.where(exposure_analysis['current_stock'] > exposure_analysis['eoq'] * 1.5, 'Medium', 'Low')
    )
    
    return exposure_analysis

# Perform exposure analysis
future_demand = best_results['forecast_future'] if best_model == 'Prophet' else best_results['forecast_future']
exposure_analysis = calculate_inventory_exposure(inventory, future_demand)

# Summary of exposure risks
print("⚠️  Inventory Exposure Risk Summary:")
print("\nStockout Risk:")
stockout_summary = exposure_analysis['stockout_risk'].value_counts()
for risk, count in stockout_summary.items():
    pct = count / len(exposure_analysis) * 100
    print(f"   {risk}: {count} items ({pct:.1f}%)")

print("\nOverstock Risk:")
overstock_summary = exposure_analysis['overstock_risk'].value_counts()
for risk, count in overstock_summary.items():
    pct = count / len(exposure_analysis) * 100
    print(f"   {risk}: {count} items ({pct:.1f}%)")

## 7. Safety Stock Optimization

In [ ]:
def optimize_safety_stock(inventory_df, service_levels=[0.90, 0.95, 0.99]):
    """Optimize safety stock for different service levels"""
    
    optimization_results = []
    
    for service_level in service_levels:
        # Calculate optimized safety stock
        # Simplified: using coefficient of variation approach
        demand_cv = 0.3  # Assumed coefficient of variation
        lead_time_avg = 14  # Average lead time in days
        
        from scipy.stats import norm
        z_score = norm.ppf(service_level)
        
        optimized_safety_stock = z_score * inventory_df['avg_demand'] * demand_cv * np.sqrt(lead_time_avg)
        
        # Calculate metrics
        current_carrying_cost = inventory_df['safety_stock'].sum() * 0.25  # 25% carrying cost
        optimized_carrying_cost = optimized_safety_stock.sum() * 0.25
        
        optimization_results.append({
            'service_level': service_level,
            'avg_safety_stock': optimized_safety_stock.mean(),
            'total_safety_stock': optimized_safety_stock.sum(),
            'carrying_cost': optimized_carrying_cost,
            'cost_change': optimized_carrying_cost - current_carrying_cost,
            'items_increased': (optimized_safety_stock > inventory_df['safety_stock']).sum(),
            'items_decreased': (optimized_safety_stock < inventory_df['safety_stock']).sum()
        })
    
    return pd.DataFrame(optimization_results)

# Optimize safety stock
safety_stock_optimization = optimize_safety_stock(inventory)

print("🎯 Safety Stock Optimization Results:")
print(safety_stock_optimization.round(2))

# Current vs optimized comparison
current_total = inventory['safety_stock'].sum()
print(f"\n📊 Current total safety stock: {current_total:.0f} units")
print(f"   Recommended (95% service): {safety_stock_optimization.loc[1, 'total_safety_stock']:.0f} units")
print(f"   Potential cost impact: ${safety_stock_optimization.loc[1, 'cost_change']:.0f}")

In [ ]:
# Visualize optimization results
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Service level vs carrying cost
axes[0,0].plot(safety_stock_optimization['service_level'], safety_stock_optimization['carrying_cost'], 'bo-')
axes[0,0].set_title('Service Level vs Carrying Cost')
axes[0,0].set_xlabel('Service Level')
axes[0,0].set_ylabel('Annual Carrying Cost ($)')
axes[0,0].grid(True)

# Exposure risk distribution
risk_matrix = pd.crosstab(exposure_analysis['stockout_risk'], exposure_analysis['overstock_risk'])
sns.heatmap(risk_matrix, annot=True, fmt='d', ax=axes[0,1], cmap='YlOrRd')
axes[0,1].set_title('Risk Matrix: Stockout vs Overstock')

# Days of supply distribution
axes[1,0].hist(exposure_analysis['days_of_supply'], bins=30, alpha=0.7, color='skyblue')
axes[1,0].axvline(x=30, color='red', linestyle='--', label='Target: 30 days')
axes[1,0].set_title('Days of Supply Distribution')
axes[1,0].set_xlabel('Days of Supply')
axes[1,0].legend()

# Forecast accuracy by model
model_names = ['ARIMA', 'Prophet']
accuracies = [arima_results['accuracy'], prophet_results['accuracy']]
axes[1,1].bar(model_names, accuracies, color=['lightcoral', 'lightgreen'])
axes[1,1].set_title('Forecast Accuracy Comparison')
axes[1,1].set_ylabel('Accuracy (%)')
axes[1,1].set_ylim(0, 100)

plt.tight_layout()
plt.show()

## 8. Key Insights & Recommendations

In [ ]:
print("=== FORECAST ACCURACY & INVENTORY OPTIMIZATION ANALYSIS ===")

print(f"\n🔮 FORECASTING PERFORMANCE:")
print(f"   Best Model: {best_model}")
print(f"   Forecast Accuracy: {best_results['accuracy']:.1f}%")
print(f"   MAPE: {best_results['mape']:.2f}%")
print(f"   Demand Volatility (CV): {total_daily_demand['quantity'].std() / total_daily_demand['quantity'].mean():.2f}")

print(f"\n⚠️  INVENTORY EXPOSURE:")
high_stockout_risk = (exposure_analysis['stockout_risk'] == 'High').sum()
high_overstock_risk = (exposure_analysis['overstock_risk'] == 'High').sum()
print(f"   High Stockout Risk: {high_stockout_risk} items")
print(f"   High Overstock Risk: {high_overstock_risk} items")
print(f"   Average Days of Supply: {exposure_analysis['days_of_supply'].mean():.1f} days")

print(f"\n🎯 SAFETY STOCK OPTIMIZATION:")
recommended_service = 0.95
opt_row = safety_stock_optimization[safety_stock_optimization['service_level'] == recommended_service].iloc[0]
print(f"   Recommended Service Level: {recommended_service*100:.0f}%")
print(f"   Current Safety Stock: {inventory['safety_stock'].sum():.0f} units")
print(f"   Optimized Safety Stock: {opt_row['total_safety_stock']:.0f} units")
print(f"   Annual Cost Impact: ${opt_row['cost_change']:.0f}")

print(f"\n💡 KEY RECOMMENDATIONS:")

if best_results['accuracy'] < 80:
    print(f"   🔴 CRITICAL: Forecast accuracy below 80%. Improve demand sensing.")
elif best_results['accuracy'] < 90:
    print(f"   🟡 HIGH: Forecast accuracy needs improvement. Consider external factors.")
else:
    print(f"   ✅ GOOD: Forecast accuracy above 90%. Maintain current approach.")

if high_stockout_risk > len(inventory) * 0.1:
    print(f"   🔴 CRITICAL: {high_stockout_risk} items at high stockout risk. Immediate action needed.")

if high_overstock_risk > len(inventory) * 0.1:
    print(f"   🟡 MEDIUM: {high_overstock_risk} items with excess stock. Review EOQ parameters.")

if abs(opt_row['cost_change']) > 10000:
    print(f"   🟡 MEDIUM: Significant safety stock optimization opportunity (${abs(opt_row['cost_change']):.0f}).")

print(f"\n📋 NEXT STEPS:")
print(f"   1. Implement {best_model} model for demand forecasting")
print(f"   2. Review {high_stockout_risk} items with high stockout risk")
print(f"   3. Optimize safety stock for 95% service level")
print(f"   4. Monitor forecast accuracy weekly and retrain models monthly")